# Air Pollution in Syria

Air Quality and, in particular, levels of fine particulate matter (e.g., PM2.5 and PM 10 ) over cities can be a proxy for assessment of economic activity and density of city populations (Yailymova et.al., 2023). NO2 along with other NOx reacts with other chemicals in the air to form both particulate matter and ozone. A 2020 study of cities in Sub-Saharan Africa finds that NO2 provides a useful, albeit “noisy”, real-time proxy measure of how COVID-19 has affected [economic activity](https://blogs.worldbank.org/developmenttalk/what-nitrogen-dioxide-emissions-tell-us-about-fragile-recovery-south-asia). Studies also found a clear relationship between NO2 levels and industrial production growth (Deb, et.al., 2020). 

## Data

The data for this was obtained from the Copernicus Satellite Imagery available through Google Earth Engine. We clipped the dataset to get it at a monthly level for each admin 2 region. Nitrogen oxides (NO2 and NO) are important trace gases in the Earth's atmosphere, present in both the troposphere and the stratosphere. They enter the atmosphere as a result of anthropogenic activities (notably fossil fuel combustion and biomass burning) and natural processes (wildfires, lightning, and microbiological processes in soils). Here, NO2 is used to represent concentrations of collective nitrogen oxides because during daytime, i.e. in the presence of sunlight, a photochemical cycle involving ozone (O3) converts NO into NO2 and vice versa on a timescale of minutes.

The total vertical column of NO2 is calculated by taking the slant column density (SCD) of NO2, which is the amount of NO2 measured along the line of sight of the satellite instrument, and dividing it by the air mass factor (AMF), which corrects the slant column density for the effect of the path length and the scattering and absorption properties of the atmosphere. The Total Vertical Column Density (VCD) is typically given in molecules per square centimeter (molec/cm²). 

## References

- Deb, P., Furceri, D., Ostry, J. D., & Tawk, N. (2020). The economic effects of COVID-19 containment measures.

- Yailymova, H., Kolotii, A., Kussul, N., & Shelestov, A. (2023, July). Air quality as proxy for assesment of economic activity. In IEEE EUROCON 2023-20th International Conference on Smart Technologies (pp. 89-92). IEEE.

In [122]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

import bokeh
from bokeh.layouts import column
from bokeh.models import Legend, TabPanel, Tabs

from bokeh.core.validation.warnings import EMPTY_LAYOUT, MISSING_RENDERERS

bokeh.core.validation.silence(EMPTY_LAYOUT, True)
bokeh.core.validation.silence(MISSING_RENDERERS, True)
from bokeh.plotting import figure, show, output_notebook

In [123]:
syria_adm0 = gpd.read_file(
    '../../data/boundaries/syr_admin0.shp'
)
syria_adm1 = gpd.read_file('../../data/boundaries/syr_admin1.shp')
syria_adm2 = gpd.read_file('../../data/boundaries/syr_admin2.shp')
syria_adm3 = gpd.read_file('../../data/boundaries/syr_admin3.shp')

In [124]:
monthly_no2_adm1 = pd.read_csv('../../data/airpollution/admin1/syria_adm1_no2_monthly_combined.csv')
monthly_no2_adm2 = pd.read_csv('../../data/airpollution/admin2/syr_admin2_no2_monthly_combined.csv')
monthly_no2_adm3 = pd.read_csv('../../data/airpollution/admin3/syr_admin3_no2_monthly_combined.csv')
monthly_no2_adm0 = pd.read_csv('../../data/airpollution/admin0/syr_admin0_no2_monthly_combined.csv')

In [125]:
monthly_no2_national = monthly_no2_adm0.copy()

In [128]:
#
# monthly_no2_national.to_csv("../../data/air_pollution/Syria_National_NO2.csv")

## Temporal Pollution Trends

Now let's create interactive visualizations using Altair that allow for zooming, panning, and hovering over data points.

In [129]:
# Import Altair and our custom visualization functions
import altair as alt
import importlib
import airpollution_visuals
importlib.reload(airpollution_visuals)  # Reload to get the latest changes
from airpollution_visuals import create_interactive_lineplot, create_multi_admin_lineplot

# Enable Altair to render in Jupyter notebooks
alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

### Interactive Features:
- **Zoom/Pan**: Click and drag to zoom into specific time periods, use scroll wheel to zoom, double-click to reset
- **Legend**: Click on legend items to hide/show specific regions (lines will become faded when deselected)
- **Hover**: Hover over data points to see detailed values
- **Multiple Selection**: Hold Shift and click legend items to select/deselect multiple regions

In [130]:
# Create an interactive line plot for national-level data
national_chart = create_interactive_lineplot(
    monthly_no2_national,
    title="National NO2 Levels in Syria",
    subtitle="Source: Sentinel-5P TROPOMI",
    categories="Syrian Arab Republic",  # Only show national level
    width=560  # 30% smaller than default 800
)

national_chart.show()

alt.Chart(...)

In [131]:
# Create an interactive plot for selected Admin 1 regions
top5 = monthly_no2_adm1.groupby('NAME_EN')['mean'].mean().nlargest(5).index.tolist()
selected_admin1_regions = top5

admin1_chart = create_interactive_lineplot(
    monthly_no2_adm1,
    title="Air Pollution Levels in Highest Polluting Syrian Provinces",
    subtitle="Source: Sentinel-5P TROPOMI",
    categories=selected_admin1_regions,
    width=560  # 30% smaller than default 800
)

admin1_chart.show()

alt.Chart(...)

In [132]:
# Create a comparison chart showing all admin1 regions
all_admin1_chart = create_interactive_lineplot(
    monthly_no2_adm1,
    title="Air Pollution Levels Across All Syrian Provinces",
    subtitle="All provinces shown - use legend to hide/show specific regions",
    width=700,  # 30% smaller than original 1000
    height=500
)

all_admin1_chart.show()

alt.Chart(...)

## Temporal Maps

Now let's create maps that show how air pollution levels change over time across different regions.

In [133]:
# Import the new mapping functions
import importlib
importlib.reload(airpollution_visuals)
from airpollution_visuals import *

In [134]:
# Import the updated mapping functions with shared legend
import importlib
importlib.reload(airpollution_visuals)
from airpollution_visuals import create_temporal_maps

# Create yearly maps with single common legend and consistent color scale
yearly_maps = create_temporal_maps(
    dataframe=monthly_no2_adm3,
    boundaries_gdf=syria_adm3,
    join_column="NAME_EN",
    width=250,
    height=150,
    color_scheme="blues"
)

yearly_maps.show()

alt.Chart(...)

In [135]:
# Create monthly maps for Damascus admin3 regions with custom title
monthly_maps_2023 = create_temporal_maps(
    dataframe=monthly_no2_adm3,
    boundaries_gdf=syria_adm3[syria_adm3['ADM1_EN']=='Damascus'],
    join_column="NAME_EN",
    title="NO2 Pollution in Damascus Districts",  # Custom title
    temporal_grouping="year",
    width=200,
    height=200
)

monthly_maps_2023.show()

alt.Chart(...)

<div class='tableauPlaceholder' id='viz1699370957260' style='position: relative'><noscript><a href='#'><img alt='No2 Levels in Syria Admin Level 2 ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ai&#47;AirPollutioninSyria&#47;Sheet1&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='AirPollutioninSyria&#47;Sheet1' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ai&#47;AirPollutioninSyria&#47;Sheet1&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1699370957260');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>